In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
import pandas as pd
import numpy as np
import matplotlib as mpl

import matplotlib.pyplot as plt
import platform
if platform.system() == "Windows":
    plt.rcParams['font.family'] = ['Times new roman',] # windows->Heiti TC
else:
    plt.rcParams['font.family'] = ['Heiti TC',] # windows->Heiti TC
    
plt.rcParams['axes.unicode_minus'] = False # windows->simhei
plt.rcParams['figure.dpi'] = 200

import geopandas as gpd
import shapely
from shapely.geometry import Point,Polygon,LineString,MultiLineString,MultiPoint,MultiPolygon

import pickle
from tqdm import tqdm
from glob import glob
import joblib

import seaborn as sns

In [2]:
import os

import platform
if platform.system() == "Windows":
    os.chdir(r'H:\BaiduSyncdisk\DR.MENG-Full\Y2024-002-DT-NANJING-ACCESSIBILITY-MAUP')
else:
    os.chdir(r'/Volumes/SANDISK/DR.MENG')

# read data

In [3]:
file_name = 'grid3000_od' # community_od grid1000_od grid500_od grid250_od, grid1500_od grid2000_od grid2500_od grid3000_od

In [4]:
od_cycle = pd.read_csv(f'./od_crawled_data_unit/Nanjing_unit_{file_name}_cycle.csv') # 骑行 7km
od_nav = pd.read_csv(f'./od_crawled_data_unit/Nanjing_unit_{file_name}_car.csv')     # 步行 5km
od_pt = pd.read_csv(f'./od_crawled_data_unit/Nanjing_unit_{file_name}_bt.csv')       # 全
od_walk = pd.read_csv(f'./od_crawled_data_unit/Nanjing_unit_{file_name}_walk.csv')

In [5]:
od_cycle.head(1) # community_od grid1000_od

,distance,duration,od_id
0,31529,9554,od_26145


In [6]:
od_nav.head(1)

,duration,distance,od_id
0,384.1,6326.3,od_18693


In [7]:
od_pt.head(1)

,all_distance,all_time,all_price,all_walk_distance,all_walk_time,access_walk_distance,egress_walk_distance,number_of_boardings,od_id,pt
0,21983,4194,7.0,1794.0,1493.0,1475.0,156.0,2.0,od_10820,1


In [8]:
od_walk.head(1)

,all_distance,all_time,od_id
0,21057,18048,od_10820


In [9]:
od_walk.rename(columns={'all_distance':'distance', 'all_time':'duration'}, inplace=True)
od_pt.rename(columns={'all_distance':'distance', 'all_time':'duration'}, inplace=True)

In [10]:
od_walk.head(1)

,distance,duration,od_id
0,21057,18048,od_10820


In [11]:
od_pt.head(1)

,distance,duration,all_price,all_walk_distance,all_walk_time,access_walk_distance,egress_walk_distance,number_of_boardings,od_id,pt
0,21983,4194,7.0,1794.0,1493.0,1475.0,156.0,2.0,od_10820,1


# select data

In [12]:
od_pt = od_pt[['od_id','distance','duration']]
od_walk = od_walk[['od_id','distance','duration']]
od_nav = od_nav[['od_id','distance','duration']]
od_cycle = od_cycle[['od_id','distance','duration']]

In [13]:
od_pt.head(1)

,od_id,distance,duration
0,od_10820,21983,4194


In [14]:
od_nav.head(1)

,od_id,distance,duration
0,od_18693,6326.3,384.1


In [15]:
od_pt.head(1)

,od_id,distance,duration
0,od_10820,21983,4194


In [16]:
od_walk.head(1)

,od_id,distance,duration
0,od_10820,21057,18048


In [17]:
len(od_cycle),len(od_nav),len(od_pt),len(od_walk)

(51726, 51726, 51557, 51726)

In [18]:
dt_full_od = pd.read_json(f'./od_request_data_unit/{file_name}.json')

In [19]:
len(dt_full_od)

51726

In [20]:
dt_full_od.head(1)

,name,park_id,entrance,net_id,lng,lat,distance,od_id
0,宝船厂遗址公园,p_0,"[118.7309901401, 32.0639718849]",404,118.778847,31.864796,22602.770662,od_0


In [21]:
dt_full_od['x_parks'] = dt_full_od['entrance'].str.get(0)
dt_full_od['y_parks'] = dt_full_od['entrance'].str.get(1)

In [22]:
dt_full_od = dt_full_od.drop(['entrance','distance'], axis=1)

In [23]:
dt_full_od.head(1)

,name,park_id,net_id,lng,lat,od_id,x_parks,y_parks
0,宝船厂遗址公园,p_0,404,118.778847,31.864796,od_0,118.73099,32.063972


# merge data

In [24]:
od_cycle.head(1)

,od_id,distance,duration
0,od_26145,31529,9554


In [25]:
dt_full_od.head(1)

,name,park_id,net_id,lng,lat,od_id,x_parks,y_parks
0,宝船厂遗址公园,p_0,404,118.778847,31.864796,od_0,118.73099,32.063972


In [26]:
od_cycle = od_cycle.merge(dt_full_od, on='od_id')
od_nav = od_nav.merge(dt_full_od, on='od_id')
od_pt = od_pt.merge(dt_full_od, on='od_id')
od_walk = od_walk.merge(dt_full_od, on='od_id')

In [27]:
od_cycle.head(1)

,od_id,distance,duration,name,park_id,net_id,lng,lat,x_parks,y_parks
0,od_26145,31529,9554,大学城生态公园,p_55,586,118.700317,32.058312,118.888304,31.905333


In [28]:
od_nav.head(1)

,od_id,distance,duration,name,park_id,net_id,lng,lat,x_parks,y_parks
0,od_18693,6326.3,384.1,新城公园,p_38,741,118.893859,32.100239,118.881356,32.132049


In [29]:
od_pt.head(1)

,od_id,distance,duration,name,park_id,net_id,lng,lat,x_parks,y_parks
0,od_10820,21983,4194,月牙湖公园,p_23,511,118.990674,32.12108,118.825773,32.032019


In [30]:
od_walk.head(1)

,od_id,distance,duration,name,park_id,net_id,lng,lat,x_parks,y_parks
0,od_10820,21057,18048,月牙湖公园,p_23,511,118.990674,32.12108,118.825773,32.032019


In [31]:
# od_cycle.to_csv(f'../data_to_cal_accessibility/od_cycle_{file_name}.csv', index=False) 
# od_nav.to_csv(f'../data_to_cal_accessibility/od_nav_{file_name}.csv', index=False) 
# od_pt.to_csv(f'../data_to_cal_accessibility/od_pt_{file_name}.csv', index=False)
# od_walk.to_csv(f'../data_to_cal_accessibility/od_walk_{file_name}.csv', index=False) 